In [34]:
from litellm import completion
import gradio as gr

In [35]:
OLLAMA_MODEL = 'ollama_chat/gpt-oss:latest'
OLLAMA_BASE_URL = 'http://localhost:11434/v1'
OLLAMA_API_KEY = "CAN_BE_ANYTHING"

In [36]:
SYSTEM_PROMPT = """
You are an expert Code Transpiler and Polyglot Engineer. Your sole purpose is to assist users with code conversion tasks.

### CORE BEHAVIOR
Analyze the user's input to determine if it is a **Conversational Query** or a **Code Conversion Request**.

#### 1. CONVERSATIONAL QUERY (No Code Provided)
*   **Scope:** Handle questions strictly related to code conversion (e.g., "Can you convert Python to Java?", "What languages do you support?").
*   **Refusal:** If the user asks about unrelated topics (e.g., "What is the capital of France?", "Write me a poem"), politely refuse and state that you are a specialized code conversion tool.
*   **Tone:** Helpful, professional, and concise.

#### 2. CODE CONVERSION REQUEST (Source Code Provided)
*   **Default Target:** If no target language is specified, convert the code to **Java**.
*   **Quality:** Refactor logic to be idiomatic, performant, and structurally complete.
*   **Formatting (CRITICAL):**
    *   Wrap the final code output in Markdown backticks (e.g., ``````) so it renders correctly in the UI.
    *   Do NOT include conversational filler like "Here is the code" outside the backticks. Just provide the code block.

### LOGIC FLOW
1.  **Check Input:** Does the input contain a block of code or a file attachment?
    *   **YES:** Execute "Code Conversion Request" protocols.
    *   **NO:** Execute "Conversational Query" protocols.
"""


In [37]:
def translate_code(message, history):
    prompt = message.get('text', '')
    filePaths = message.get('files', [])
    
    messages = [{"role": "system", "content": SYSTEM_PROMPT}]
    messages.extend(history)

    user_content = [f"User Instructions: {prompt}"]

    if filePaths:
        try:
            with open(filePaths[0], 'r') as data:
                content = data.read()
            user_content.append(f"Source code:\n\n{content}")
        except Exception as e:
            pass
    
    final_query = '\n\n'.join(user_content)

    messages.append({"role": "user", "content": final_query})

    print(messages)

    stream = completion(
        model = OLLAMA_MODEL,
        messages = messages,
        stream = True
    )

    response = ''
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
gr.ChatInterface(
    fn = translate_code, 
    textbox = gr.MultimodalTextbox(placeholder = 'Type your message..'), 
    multimodal = True,
    type = 'messages'
).launch()

* Running on local URL:  http://127.0.0.1:7869
* To create a public link, set `share=True` in `launch()`.


[{'role': 'system', 'content': '\nYou are an expert Code Transpiler and Polyglot Engineer. Your sole purpose is to assist users with code conversion tasks.\n\n### CORE BEHAVIOR\nAnalyze the user\'s input to determine if it is a **Conversational Query** or a **Code Conversion Request**.\n\n#### 1. CONVERSATIONAL QUERY (No Code Provided)\n*   **Scope:** Handle questions strictly related to code conversion (e.g., "Can you convert Python to Java?", "What languages do you support?").\n*   **Refusal:** If the user asks about unrelated topics (e.g., "What is the capital of France?", "Write me a poem"), politely refuse and state that you are a specialized code conversion tool.\n*   **Tone:** Helpful, professional, and concise.\n\n#### 2. CODE CONVERSION REQUEST (Source Code Provided)\n*   **Default Target:** If no target language is specified, convert the code to **Java**.\n*   **Quality:** Refactor logic to be idiomatic, performant, and structurally complete.\n*   **Formatting (CRITICAL):**\n